In [1]:
# cnn model
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
import os
from sliding_window import sliding_window

In [2]:
# Hardcoded number of sensor channels employed in the gyro X challenge - X,Y,Z
SENSOR_CHANNELS = 3
# Hardcoded number of classes in the gesture recognition problem
NUM_CLASSES = 12
# Hardcoded length of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_LENGTH = 24
# Hardcoded step of the sliding window mechanism employed to segment the data
SLIDING_WINDOW_STEP = 12

# current path
cwd = os.getcwd()

In [3]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [4]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

In [5]:
def load_dataset_group(dataset_type):
    path = cwd+'/{}data.txt'.format(dataset_type)
    data = load_file(path)
    return data[:,0:3], data[:,-1]

In [6]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train')
    print("Train set:" + str(trainX.shape) + str(trainy.shape))
    # load all test
    testX, testy = load_dataset_group('test')
    print("Test set:" + str(testX.shape) + str(testy.shape))

    return trainX, trainy, testX, testy

In [7]:
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

In [8]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    _verbose, epochs, batch_size = 1, 10, 100
    #n_timesteps, n_features, n_outputs = trainX.shape[0], trainX.shape[1], 12
    n_timesteps, n_features, n_outputs = 24, 3, 12
    print(n_timesteps, n_features, n_outputs)
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=_verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=_verbose)
    return accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [9]:
# run an experiment
def run_experiment(repeats=1):
    # load data
    trainX, trainy, testX, testy = load_dataset()

    # Sensor data is segmented using a sliding window mechanism
    trainX, trainy = opp_sliding_window(trainX, trainy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
    testX, testy = opp_sliding_window(testX, testy, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)

    # Data is reshaped
    trainX = trainX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D
    testX = testX.reshape((-1, SLIDING_WINDOW_LENGTH, SENSOR_CHANNELS)) # for input to Conv1D

    #convert data to categorical form [0 0 0 0 0 0 0 0 1 0 0 0 0]. Represent class in vector
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    
    print(trainy[0])
    print(" ..after sliding and reshaping, train data: inputs {0}, targets {1}".format(trainX.shape, trainy.shape))
    print(" ..after sliding and reshaping, test data : inputs {0}, targets {1}".format(testX.shape, testy.shape))

    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

# run the experiment
run_experiment()

Train set:(273942, 3)(273942,)
Test set:(125368, 3)(125368,)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 ..after sliding and reshaping, train data: inputs (22827, 24, 3), targets (22827, 12)
 ..after sliding and reshaping, test data : inputs (10446, 24, 3), targets (10446, 12)
24 3 12
Epoch 1/10
229/229 [==============================] - 1s 3ms/step - loss: 1.7294 - accuracy: 0.3864
Epoch 2/10
229/229 [==============================] - 1s 3ms/step - loss: 1.5398 - accuracy: 0.4556
Epoch 3/10
229/229 [==============================] - 1s 3ms/step - loss: 1.4953 - accuracy: 0.4705
Epoch 4/10
229/229 [==============================] - 1s 3ms/step - loss: 1.4604 - accuracy: 0.4828
Epoch 5/10
229/229 [==============================] - 1s 3ms/step - loss: 1.4259 - accuracy: 0.5024
Epoch 6/10
229/229 [==============================] - 1s 3ms/step - loss: 1.4008 - accuracy: 0.5061
Epoch 7/10
229/229 [==============================] - 1s 3ms/step - loss: 1.3607 - accuracy: 0.5198
Epoch 8/10
229/229 